<a href="https://colab.research.google.com/github/zlximon/Tensorflow_DL/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#downloading helper functions 
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2022-06-13 18:11:17--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-06-13 18:11:17 (69.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys



#get text dataset

In [3]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-06-13 18:11:20--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.135.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.003s  

2022-06-13 18:11:20 (167 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



  #visualizing text dataset

In [4]:
import pandas as pd 

In [5]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()
# 1 is for disaster 

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
len(train_df),len(test_df)

(7613, 3263)

In [10]:
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Annihilated Abs . ?? http://t.co/1xPw292tJe

---

Target: 1 (real disaster)
Text:
How One Couple Is Using Drones to Save Refugees at the World's Deadliest Border http://t.co/9qpG0Z3Rh9

---

Target: 0 (not real disaster)
Text:
Top link: Reddit's new content policy goes into effect many horrible subreddits banned or quarantined http://t.co/BjVfk1ETe9

---

Target: 1 (real disaster)
Text:
Trafford Centre film fans angry after Odeon cinema evacuated following false fire alarm: 
 Twitter users tell ... http://t.co/dZLENSe1Gw

---

Target: 0 (not real disaster)
Text:
Tennis: Defending champ Svetlana Kuznetsova withdraws from Citi Open cites injury to her lower left leg (ESPN) http://t.co/iM2HdsKlq5

---



#split data into training and validation  sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),train_df_shuffled["target"],test_size=0.1,
                                                                       random_state=42)

In [13]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [14]:
train_sentences[:10],train_labels[:10 ]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), 5921    0


Embedding : create a matrix of features vector for each token 

#text vectorization ( tokenization)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [16]:
text_vectorizer=TextVectorization(max_tokens=None# how many word in vocalubary  and None means there is no cap 
                                  ,standardize="lower_and_strip_punctuation",# turn all words to lower case and strip 
                                  split="whitespace",ngrams=None,output_mode="int",output_sequence_length=None,# how long do we want sequences be 
                                 )# pad_to_max_tokens=True)#add zeros into end of short lenght 

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None # how long do you want your sequences to be?
                                    )

In [18]:
#find the average numbers of tokens
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [19]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [20]:
# fit the text vectorization to the training text
text_vectorizer.adapt(train_sentences)


In [21]:
#create a sample 
sample="There is a flood in my street"
text_vectorizer([sample])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
#choose a random sentence from training dataset
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
You Are Invited to The Expo Explosion Summer Event 2015! 
WHEN: August 14th Friday 2015
WHERE: Ben E Keith... http://t.co/yh4R7Ug21a        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  12,   22,    1,    5,    2,    1,  303,  270, 1414,  208,   45,
         407,    1, 1208,  208]])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # the most common words in the vocab
bottom_5_words = words_in_vocab[-5:] # the least common words in the vocab
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab: {bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#creating Embedding layer 

Embedding : turns positives integers into dense vectors of fixed size
parameters for our embedding

1-input_dim - The size of the vocabulary

2-output_dim - The size of the output embedding vector, for example, a value of 100 outputs a feature vector of size 100 for each word.

3-embeddings_initializer - How to initialize the embeddings matrix, default is "uniform" which randomly initalizes embedding matrix with uniform distribution. This can be changed for using pre-learned embeddings.

4- input_length - Length of sequences being passed to embedding layer.


In [24]:
from tensorflow.keras import layers 
embedding =layers.Embedding(input_dim=max_vocab_length,output_dim=128,input_length=max_length)
embedding

In [25]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Enugu Government to demolish illegal structures at International Conference Centre http://t.co/7K5SHaiqIw      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01884684,  0.02696272, -0.01584379, ..., -0.00589123,
          0.03029266,  0.03957856],
        [ 0.03668525,  0.0267717 , -0.00787092, ..., -0.00952761,
         -0.00042444,  0.00785685],
        [ 0.00117733, -0.04736103, -0.03554484, ...,  0.03049033,
         -0.00940503,  0.01290587],
        ...,
        [-0.02181221,  0.02301497, -0.04082551, ..., -0.00040898,
         -0.04431156,  0.00090283],
        [-0.02181221,  0.02301497, -0.04082551, ..., -0.00040898,
         -0.04431156,  0.00090283],
        [-0.02181221,  0.02301497, -0.04082551, ..., -0.00040898,
         -0.04431156,  0.00090283]]], dtype=float32)>

In [26]:
sample_embed[0][0],sample_embed[0][0].shape,random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.01884684,  0.02696272, -0.01584379,  0.0117554 , -0.00372159,
        -0.00798015,  0.04847929, -0.04045653, -0.00635309, -0.04850348,
        -0.034807  , -0.01738437, -0.02246764, -0.01579106, -0.01193502,
         0.00270779,  0.02601829, -0.04360687, -0.028097  ,  0.01605788,
         0.0369969 , -0.01129897, -0.03330026, -0.04759187,  0.04271963,
         0.01368101, -0.02333018,  0.00392528, -0.04660982, -0.03589554,
        -0.00606322,  0.04876616,  0.01146823,  0.01119342, -0.00872419,
        -0.03711385,  0.02991282, -0.01404232, -0.03732954,  0.02112019,
        -0.04103728, -0.02919494,  0.03902255,  0.00090827,  0.00475739,
        -0.00854685, -0.00779395,  0.00218571, -0.02736499,  0.00631645,
         0.00602809,  0.04530957, -0.03736953,  0.04199325,  0.01484165,
        -0.01999544, -0.01886343,  0.00924256, -0.03898531,  0.00023588,
         0.02659888, -0.04935409, -0.00209694,  0.01263187,  0.02165165,
  

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


In [32]:
model0=Pipeline([
("tfidf",TfidfVectorizer()),#convert words to numbers using tfdif
("clf",MultinomialNB())#model the text
])
#fit pipeline to train data
model0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [34]:
baseline_score=model0.score(val_sentences,val_labels)# evaluate 
baseline_score # accuracy 

0.7926509186351706

In [37]:
baseline_pred=model0.predict(val_sentences)
baseline_pred[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])